In [6]:
import pandas as pd
import requests
import json 
import os
from dotenv import load_dotenv

def snoogums():
    """
    function that takes the results from thecatapi call and puts them into a format that's easier to read.
    the purpose for this is so that a prompt engineer can call this api and easily make out the information
    it is providing so they can write quicker, more in-depth prompts.
    """

    load_dotenv()
    
    cat_apikey = os.environ.get("cat_apikey")
    
    url = "https://api.thecatapi.com/v1/images/search?size=med&mime_types=jpg&format=json&has_breeds=true&order=RANDOM&page=0&limit=10"
    payload = {}
    headers = {
        "Content-Type": "application/json",
        "x-api-key": cat_apikey  # Make sure to include YOUR actual API key
    }
    response = requests.get(url, headers=headers, data=payload)

    # served as a checkpoint to see if the call was working
    # print(response.text)

    try:
        call_response = response.json()
    except json.JSONDecodeError as e:
        print(f"There was an error in deconding JSON: {e}")
        print(f"Response text: {response.text}")

        # exit exception block if the json decoding fails
        
        return None 

    api_df = pd.DataFrame(call_response)
    
    # good place to check how things are working
    # print(api_df)

    # Extract the 'breeds' column into a new DataFrame, applying pd.Series to turn each dictionary in the list into its own series
    breeds_df = api_df["breeds"]

    # google's gemini helped in the following code:
    # it mentioned that there may be times when multiple breeds are in a single entry
    # and what to do if that happens. didn't catch that myself, and credit where credit's due!
    if any(breeds_df.apply(len) > 1):
        breeds_df = breeds_df.explode(column=0) # Explode specific column

    breeds_df = pd.json_normalize(breeds_df)

    # now that 'breeds' has been cleaned up, we can join (concatenate)
    # it with the api_df, from which 'breeds' was removed
    api_df = pd.concat([api_df.drop('breeds', axis=1), breeds_df], axis=1)


    print("Original DataFrame with Flattened Breeds:")
    print(api_df.head().to_string(index=False, line_width=1000))

    return api_df # Returns the modified dataframe

In [7]:
snoogums()

Original DataFrame with Flattened Breeds:
       id                                             url  width  height  \
HVP1wfZF0 https://cdn2.thecatapi.com/images/HVP1wfZF0.jpg   1600    1067   
-y_6SBPrw https://cdn2.thecatapi.com/images/-y_6SBPrw.jpg   1280     959   
-tm9-znzl https://cdn2.thecatapi.com/images/-tm9-znzl.jpg   1125     750   
KUcr7eFlM https://cdn2.thecatapi.com/images/KUcr7eFlM.jpg   2405    1549   
Ttk_tdV4g https://cdn2.thecatapi.com/images/Ttk_tdV4g.jpg   1152     768   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

,id,url,width,height,0
0,HVP1wfZF0,https://cdn2.thecatapi.com/images/HVP1wfZF0.jpg,1600,1067,"{'id': 'acur', 'name': 'American Curl', 'cfa_u..."
1,-y_6SBPrw,https://cdn2.thecatapi.com/images/-y_6SBPrw.jpg,1280,959,"{'id': 'cymr', 'name': 'Cymric', 'vetstreet_ur..."
2,-tm9-znzl,https://cdn2.thecatapi.com/images/-tm9-znzl.jpg,1125,750,"{'id': 'jbob', 'name': 'Japanese Bobtail', 'cf..."
3,KUcr7eFlM,https://cdn2.thecatapi.com/images/KUcr7eFlM.jpg,2405,1549,"{'id': 'norw', 'name': 'Norwegian Forest Cat',..."
4,Ttk_tdV4g,https://cdn2.thecatapi.com/images/Ttk_tdV4g.jpg,1152,768,"{'id': 'siam', 'name': 'Siamese', 'cfa_url': '..."
5,_DFVS4T4I,https://cdn2.thecatapi.com/images/_DFVS4T4I.jpg,1080,1224,"{'id': 'sibe', 'name': 'Siberian', 'cfa_url': ..."
6,wUxNIew6I,https://cdn2.thecatapi.com/images/wUxNIew6I.jpg,1280,1920,"{'id': 'soma', 'name': 'Somali', 'cfa_url': 'h..."
7,oR3LMBqEZ,https://cdn2.thecatapi.com/images/oR3LMBqEZ.jpg,750,937,"{'id': 'tang', 'name': 'Turkish Angora', 'cfa_..."
8,PlvdElgfk,https://cdn2.thecatapi.com/images/PlvdElgfk.jpg,1080,1080,"{'id': 'ycho', 'name': 'York Chocolate', 'temp..."
9,QAWwhRFCG,https://cdn2.thecatapi.com/images/QAWwhRFCG.jpg,900,675,"{'id': 'sibe', 'name': 'Siberian', 'cfa_url': ..."
